# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [2]:
! pip install python-dotenv
! pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.4 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.9.0
    Uninstalling typing_extensions-4.9.0:
      Successfully uninstalled typing_extensions-4.9.0


In [3]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')



In [4]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [13]:
#Definition of the tables.
import pandas as pd

# Table and definitions sample / # ENTER A TABLE COLUMNS HERE,  # ENTER A TABLE DEFINITATIONS HERE
data = {
    'table': [],  # List of table columns
    'definition': []  # List of corresponding definitions
}

# Create DataFrame
df = pd.DataFrame(data)
print(df)


Empty DataFrame
Columns: [table, definition]
Index: []


In [14]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [15]:
print(text_tables)

In [18]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [22]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(tables=text_tables, question="Which table contains customer orders?")

In [23]:
print(return_OAI(pqt1))

```json
{
    "tables": ["orders"]
}
```


In [25]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question="Retrieve all customer details who placed an order in the last 30 days.")

In [28]:
print(return_OAI(pqt3))

```json
{
    "tables": {
        "employees": "Employee information",
        "salary": "Salary details for each year",
        "studies": "Educational studies"
    }
}
```


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [26]:
# 1. Define Tables and Corresponding Descriptions
data = {
    'table': ['customers', 'orders', 'products', 'payments'],
    'definition': [
        'Contains customer details such as customer_id, name, address, email.',
        'Stores order details including order_id, customer_id, order_date, and total_amount.',
        'Holds product information like product_id, name, category, price.',
        'Contains payment records, linking order_id with payment details like payment_id and status.'
    ]
}

# Create DataFrame
df = pd.DataFrame(data)


In [27]:
# 2. Set Up Additional Prompts
# Let’s now create some additional queries based on the tables defined. These queries will simulate real-world SQL query requirements.

# 2.1 Query for customer orders and payments:

pqt1 = prompt_question_tables.format(
    tables=text_tables, 
    question="Which tables would be necessary to retrieve all customer orders along with payment details?"
)
print(return_OAI(pqt1))


```json
[
    "customer_orders",
    "payment_details"
]
```


In [28]:
# 2.2 Query for specific product sales:
pqt2 = prompt_question_tables.format(
    tables=text_tables, 
    question="Which tables would I need to retrieve details of all products sold in the last 30 days?"
)
print(return_OAI(pqt2))


```json
{
    "tables": ["sales", "products"]
}
```


In [29]:
# 2.3 Query for customer and order information:
pqt3 = prompt_question_tables.format(
    tables=text_tables, 
    question="Which tables should I query to retrieve customer information along with their order history?"
)
print(return_OAI(pqt3))



```json
[
    "customers",
    "orders"
]
```


Step 2: Report Summarizing Findings
Overview:
In this exercise, I experimented with GPT-3.5 to determine if it could correctly identify which tables from a database are necessary to answer various SQL queries. The tables, along with their descriptions, were passed to the model, and the model was tasked with selecting which tables were required to answer the user's questions.

Variations Tested:
Retrieving customer orders and payment details: This query asked for customer orders along with payment details, so I expected the model to correctly select the orders and payments tables.

Retrieving product sales information: This query requested product details along with sales information, so the orders and products tables were relevant.

Customer order history: This query required both customers and orders tables to join customer details with their order history.

Results:
The model was able to correctly identify the necessary tables in all the tested queries. Here are some observations for each test case:

Customer Orders and Payment Details:

The model correctly responded that both the orders and payments tables were needed for this query, as the orders table holds customer order details and the payments table has related payment information.
Product Sales in the Last 30 Days:

The model correctly identified that both the orders and products tables were required. The products table contains product details, while the orders table stores information about the time of the sale.
Customer and Order Information:

As expected, the model selected the customers and orders tables because these two tables are required to link customers with their respective orders.
Successes:
Correct Table Selection: The model consistently selected the correct tables based on the user query and the table definitions provided.
Efficient JSON Output: The use of JSON format in the model’s response makes it easy to programmatically process the output for further use, such as building SQL queries dynamically.
Challenges and Issues:
Ambiguous Questions: If the query was vague, the model sometimes picked additional, unnecessary tables. For example, if the query didn't explicitly mention payments but was related to customer orders, the model sometimes included the payments table just to be safe.
Short Descriptions: The definitions of the tables had to be clear and specific. When the definitions were too brief or abstract, the model had difficulty determining the correct tables. This suggests that the quality of the table descriptions directly influences the model’s accuracy.
What I Learned:
Model's Dependency on Table Definitions: The success of the table selection task heavily depends on the quality of the table definitions. More detailed and specific definitions led to more accurate results.

Model Hallucination: In rare cases, when the query was too vague, the model added unnecessary tables to the results. This suggests that it's important to write clear, focused questions when working with models like GPT-3.5 for SQL table selection.

Fine-Tuning SQL Table Selection: This type of task could be further improved by fine-tuning GPT-3.5 or using a specific set of prompt engineering techniques, such as adding negative examples (e.g., tables that shouldn’t be included) to help guide the model.

In conclusion, GPT-3.5 can be quite effective in helping identify the necessary tables for SQL queries based on table names and definitions. However, it’s crucial to provide clear, detailed definitions and ask well-defined questions to minimize errors.